In [ ]:
import pickle
import pandas
import datetime
import numpy
import datetime

from matplotlib import pyplot
from matplotlib import dates
from scipy.spatial import Voronoi, voronoi_plot_2d
from statsmodels.distributions.empirical_distribution import ECDF
from scipy import misc

from matplotlib import cm
from matplotlib import pyplot
from numpy import linalg
from numpy import random
from random import sample
from scipy import signal 
from scipy import stats
from shapely.geometry import Point, Polygon
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict

import dmr
import datetime
import foursquare
import geopandas
import math
import numpy
import pandas
import pickle
import pymongo
import time
import seaborn

In [ ]:
sq_leaf = (5.0, 3.0)

In [ ]:
def leaf(name):
    fig.savefig('lab_plot/'+name+'.eps', dpi=150, format='eps', transparent=True, pad_inches=0.10)

# DMR Snapshot

In [ ]:
col = {45:'blue', 55:'purple', 65:'red', 75:'orange', 85:'yellow'}

In [ ]:
lda = {45:None, 55:None, 65:None, 75:None, 85:None}
for i in lda:
    lda[i] = pickle.load(open('kato_dmr/'+str(i)+'lda.pkl', 'rb'))

In [ ]:
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
for i in lda:
    tmp = lda[i].z_m_n
    t = numpy.array(tmp).ravel()
    y, x = numpy.histogram(t, numpy.arange(0, max(t)+2, 1))
    ax.plot(x[:-1], y, marker='.',ms=5.0, linestyle='-', color=col[i])

### Disto delta-T

In [ ]:
next_df = pickle.load(open('archive/bikemi_dataframe.pkl', 'rb'))

In [ ]:
tmp = numpy.array(next_df['Durata_sec'].tolist())
a = tmp/60

In [ ]:
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
eps = 0.5
y, x = numpy.histogram(a, numpy.arange(0, max(a)+2*eps, eps))
ax.plot(x[:-1], y, marker='.',ms=0.0, linewidth=2.4, linestyle='-', color='purple')
ax.set_xlim(3, 60)
ax.axes.get_yaxis().set_visible(False)
ax.axvline(x=numpy.mean(a), color='blue', linewidth=1.2)
ax.axvline(x=numpy.median(a), color='red', linewidth=1.2)
ax.text(16, 300000,'$\mu$')
ax.text(6, 30000,'$0.5p$')

In [ ]:
leaf('delta-time-distro')

In [ ]:
numpy.mean(a), numpy.percentile(a, 50), numpy.std(a), numpy.where(y == max(y))[0][0]*eps

In [ ]:
y[:30]

In [ ]:
numpy.percentile(a, 25), numpy.percentile(a, 75)

In [ ]:
# CCDF
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
ecdf = ECDF(a)
ax.plot(ecdf.x, 1-ecdf.y, marker='.',ms=0.5, linestyle='-', color='black')
ax.set_xlim(0, 60)
#ax.set_ylim(0, 250)

In [ ]:
ecdf(10)-ecdf(3)

### Partenze e arrivi

In [ ]:
def nit(t):
    return 60*(60*t.hour+t.minute)+t.second

In [ ]:
a = [nit(i) for i in next_df['Data_prelievo'].tolist()]
b = [nit(i) for i in next_df['Data_arrivo'].tolist()]

In [ ]:
a_w = next_df[next_df['Festivo_feriale_prelievo'] == 0]['Data_prelievo'].tolist()
a_h = next_df[next_df['Festivo_feriale_prelievo'] == 1]['Data_prelievo'].tolist()
b_w = next_df[next_df['Festivo_feriale_prelievo'] == 0]['Data_arrivo'].tolist()
b_h = next_df[next_df['Festivo_feriale_prelievo'] == 1]['Data_arrivo'].tolist()

In [ ]:
fig = pyplot.figure(figsize=(20,10))
ax = fig.gca()
eps = 10*60
sh = 7*60*60

y, x = numpy.histogram([nit(i) for i in a_w], numpy.arange(0, 86400+eps, eps))
y = numpy.roll(y, int(-sh/eps))
ax.plot(x[:-1], y, marker='o',ms=0.0, linewidth=1.75, linestyle='--', color='green')

y, x = numpy.histogram([nit(i) for i in a_h], numpy.arange(0, 86400+eps, eps))
y = numpy.roll(y, int(-sh/eps))
ax.plot(x[:-1], y, marker='o',ms=0.0, linewidth=1.75, linestyle='--', color='blue')

y, x = numpy.histogram([nit(i) for i in b_w], numpy.arange(0, 86400+eps, eps))
y = numpy.roll(y, int(-sh/eps))
ax.plot(x[:-1], y, marker='o',ms=0.0, linewidth=1.75, linestyle='--', color='red')

y, x = numpy.histogram([nit(i) for i in b_h], numpy.arange(0, 86400+eps, eps))
y = numpy.roll(y, int(-sh/eps))
ax.plot(x[:-1], y, marker='o',ms=0.0, linewidth=1.75, linestyle='--', color='yellow')

ax.set_xlim(0, 18*60*60)
ax.axes.get_yaxis().set_visible(False)

In [ ]:
t1 = [l.get_text() for l in ax.get_xticklabels()]
t2 = [int((int(i)+sh)%86400/60/60) for i in t1]
ax.set_xticklabels(t2)

In [ ]:
leaf('partenza-arrivo-distro')

## Voronoi Plots 

In [ ]:
gps = pickle.load(open('/home/datasets/bikemi/station_gps_location.pkl', 'rb'))

In [ ]:
lst = [(g[1],g[0]) for g in gps.values()]
raw_voronoi = Voronoi(lst)

In [ ]:
# figur-tela e i sui assi
fig = pyplot.figure(figsize=sq_leaf)

In [ ]:
ax = fig.gca()

In [ ]:
voronoi_plot_2d(
    raw_voronoi,
    ax = ax,
    show_points = True,
    show_vertices = False,
    line_colors = "black",
    line_width = 1.0,
    line_alpha = 1.0,
    point_size = 3.5
)

In [ ]:
leaf('voronoi-galaxy')

In [ ]:
# figur-tela e i sui assi
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()

cy = [i[1] for i in raw_voronoi.vertices]
cx = [i[0] for i in raw_voronoi.vertices]

ax.plot(cx, cy, marker='.',ms=1.5, linestyle=' ', color='purple')

ax.set_xlim(min(cx)-0.04,max(cx)+0.04)
ax.set_ylim(min(cy)-0.04,max(cy)+0.04)

### Sample 

In [ ]:
csv_uri = '/home/datasets/bikemi/2017/12 2017.csv'

In [ ]:
df = pandas.read_csv(csv_uri,
                           lineterminator ='\r',
                           encoding = 'iso8859_2',
                           sep = ';',
                           parse_dates = ['Data_prelievo','Data_arrivo'],
                           date_parser = lambda x: datetime.datetime.strptime(x,'%d/%m/%y %H:%M'),
                           decimal = '.'
                          )

In [ ]:
nomi = set()
for _, i in df.iterrows():
    nomi.add(i['Nome_stazione_arrivo'])
    nomi.add(i['Nome_stazione_prelievo'])

In [ ]:
nomi

In [ ]:
df.columns

### Beta distro

In [ ]:
# distro word topic
rgb = ['black','grey','pink','brown','purple','blue','green','yellow','orange','red']
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
for i in range(len(categories)):
    t= [b[i,j] for j in range(len(b[i]))]
    x, y = numpy.histogram(t, range(max(t)))
    ax.loglog(y[1:], x, marker='.',ms=2.25, linestyle='', color=rgb[i])